In [1]:
# Gravitational Lensing Research
# Program to take a qlens-generated sample chain and derive alternative parameters
# d5: This version takes the corecusp format (k0, a, s) and derives mass, NEW concentration 
# and core radius, (mvir, c_hat_200, r_core)
# where c_hat is defined as r_vir divided by the radius at which the log slope is -2,
# and r_core is defined as the radius at which the log slope is -1.
# Uses python 3.6
# n fixed to 3

In [2]:
import time
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import pandas as pd
import timeit
from IPython.core.debugger import set_trace
import datetime
from shutil import copyfile
from astropy import constants as const
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s /u.Mpc, Om0=0.3)
print(cosmo)
from scipy.integrate import quad, dblquad
from scipy.optimize import minimize, minimize_scalar
from numba import jit
%precision %.5g

FlatLambdaCDM(H0=70 km / (Mpc s), Om0=0.3, Tcmb0=0 K, Neff=3.04, m_nu=None, Ob0=None)


'%.5g'

In [3]:
import multiprocessing as mp
nproc = 3

In [4]:
# The current working directory should CONTAIN the "chains_..." subdirectory for the run of interest.
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/Python


In [5]:
label = 'A2537.cc.v9'
newflag = False# Use this to account for the ".new" in names.
if newflag:
    extra = ".new"
else:
    extra = ''
rootpath = '/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/A2537' 
# rootpath = '/home/kea/KEVIN/UCI/Research/gravlensing/A383'
os.chdir(rootpath)
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/A2537


In [6]:
# verify that the key files exist
assert os.path.exists('chains_' + label)
for ext in ['', '.nparam', '.paramnames', '.latex_paramnames', '.ranges']:
    assert os.path.exists('chains_' + label + '/' + label + extra + ext)

In [7]:
path = rootpath + '/chains_' + label
label = label + extra

In [8]:
# create the new directory, if it doesn't already exist
app = '_d7'
newlabel = label + app
if not os.path.exists(os.getcwd() + '/chains_' + newlabel):
        os.makedirs(os.getcwd() + '/chains_' + newlabel)

In [9]:
# copy key files
for ext in ['', '.nparam', '.paramnames', '.latex_paramnames', '.ranges']:
    copyfile(path + '/' + label + ext, 'chains_' + newlabel + '/' + newlabel + ext) 

In [10]:
# create 'newpath', for convenience
newpath = rootpath + '/chains_' + newlabel

In [11]:
# we are deriving scale radius and core radius
addl_pnames = ['m_vir', 'c_hat_200', 'r_core', 'rm2']
addl_latex_pnames = ['m_{vir}', '\hat{c}_{200}', 'r_{core}', 'r_{-2}']

In [12]:
zlens = 0.189
zsrc = 2.0  # try to consistently use this

In [13]:
rho_crit = cosmo.critical_density(zlens).to(u.M_sun / u.kpc**3)
print(rho_crit)
kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(zlens) * u.arcmin / (60. * u.arcsec)
print (kpc_per_arcsec)
rho_200 = rho_crit * 200 * kpc_per_arcsec**3  # units of M_sun arcsec^-3
print('rho_200 = ', rho_200)
# get rid of units now
rho_200 = rho_200.value

163.77280884946157 solMass / kpc3
3.1556470486647226 kpc / arcsec
rho_200 =  1029288.3583223283 solMass / arcsec3


$\Sigma _{\text{crit}}=\frac{c^2 D_S}{4 \pi  G D_L D_{\text{SL}}}$

In [14]:
sigma_crit = (const.c**2 * cosmo.angular_diameter_distance(zsrc) / (4. * np.pi * const.G *\
                           cosmo.angular_diameter_distance(zlens) * cosmo.angular_diameter_distance_z1z2(zlens, zsrc))\
             ).to(u.M_sun / u.kpc**2)
print(sigma_crit)   
# get rid of units now
sigma_crit = sigma_crit.value

3003559185.4506283 solMass / kpc2


In [15]:
# read in the chain and append the new parameter names
samples = np.loadtxt(newpath + '/' + newlabel, comments="#", delimiter=None, unpack=False)
with open(newpath + '/' + newlabel + '.paramnames') as afile:
    paramnames = afile.readlines()
with open(newpath + '/' + newlabel + '.latex_paramnames') as afile:
    latex_paramnames = afile.readlines()
    

In [16]:
# special fix for Abell_611_v28.cc: change parameter name for perturber 2
# for i in range(len(paramnames)):
#     if paramnames[i].find('mass2d(100000)') != -1:
#         paramnames[i] = 'mass_p2\n'
# with open(newpath + '/' + newlabel + '.paramnames', 'w') as afile:
#     afile.write(''.join(paramnames) + '')
    
print(paramnames)
    
for i in range(len(latex_paramnames)):
    if latex_paramnames[i].find('mass2d(100000)') != -1:
        latex_paramnames[i] = 'mass_p2\tM_{p2}\n'
        
with open(newpath + '/' + newlabel + '.latex_paramnames', 'w') as afile:
    afile.write(''.join(latex_paramnames) + '')

print(latex_paramnames)



['k0\n', 'gamma\n', 'a\n', 's\n', 'q\n', 'theta\n', 'xc\n', 'yc\n', 'shear1\n', 'shear2\n', 'mtot1\n', 'mtot2\n', 'raw_chisq\n']
['k0\t\\kappa_{0}\n', 'gamma\t\\gamma\n', 'a\ta\n', 's\ts\n', 'q\tq\n', 'theta\t\\theta\n', 'xc\tx_{c}\n', 'yc\ty_{c}\n', 'shear1\t\\gamma_{1}\n', 'shear2\t\\gamma_{2}\n', 'mtot1\tM_{tot,1}\n', 'mtot2\tM_{tot,2}\n', 'raw_chisq\t\\chi^2\n']


In [17]:
# append the new parameter names
for nm, latex_nm in zip(addl_pnames, addl_latex_pnames):
    paramnames.append(nm + '\n')
    latex_paramnames.append(nm + '\t' + latex_nm + '\n')
    with open(newpath + '/' + newlabel + '.paramnames', 'a') as afile:
        afile.write(nm + '\n')
    with open(newpath + '/' + newlabel + '.latex_paramnames', 'a') as afile:
        afile.write(nm + '\t' + latex_nm + '\n')

In [18]:
# change the working directory to the new path
os.chdir(newpath)
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/A2537/chains_A2537.cc.v9_d7


In [19]:
# increase the number of derived parameters in the .nparam file
pnum, dpnum = np.loadtxt(newlabel + '.nparam', comments="#", delimiter=None, unpack=False)
addl_np = len(addl_pnames)
print(dpnum, ' derived parameters, adding ', addl_np)
dpnum += addl_np
np.savetxt(newlabel + '.nparam', [np.int(pnum), np.int(dpnum)], fmt='%i')

# add new ranges to the .ranges file. Not sure if these should be calculated, just using 0 and 1e30.
with open(newlabel + '.ranges', 'a') as afile:
    for nm in addl_pnames:
        afile.write('0  1e30\n')

# Strip off the \n character.
paramnames = [item[:-1] for item in paramnames] 
latex_paramnames = [item[:-1] for item in latex_paramnames]  
samples = np.array(samples)

1.0  derived parameters, adding  4


In [20]:
ns, p = samples.shape # note that 'p' here is two more than the number of parameters.
# as it counts the first column (weight) and the last (chisquare).
print(samples.shape)
print(paramnames)

(120367, 15)
['k0', 'gamma', 'a', 's', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2', 'mtot1', 'mtot2', 'raw_chisq', 'm_vir', 'c_hat_200', 'r_core', 'rm2']


In [21]:
# extend the samples to have addl_np additional columns
samples2 = np.append(samples, np.zeros((ns, addl_np)), axis=1)

In [22]:
# move the chisquare to be the last column
samples2[:,-1] = samples2[:, p-1]

We can use the equation for virial mass and solve for the virial radius:
$m_{200} = \frac{4}{3} \pi r_{200}^3 \rho_{200}$, and $r_s = r_{200} / c$,
Therefore
$$ r_s = (\frac{3}{4\pi})^{1/3} (m_{200} / (c  \rho_{200}))^{1/3}$$

In [23]:
(3 / (np.pi * 4.))**(1./3)

0.62035

In [24]:
# calculate the new parameters
[k0, gamma, r_s, r_c, q] = [col for col in samples2[:, [1,2, 3, 4, 5]].T] 
# for the cc model with n fixed, we need cols [1,2, 3, 4, 5], which are k0, gamma, a1, s and q

In [25]:
# import mpmath
# def AppellF1(a, b, c, d , x, y):
#     return mpmath.hyper2d({'m+n':[a], 'm':[b], 'n':[c]}, {'m+n':[d]}, x, y)
# # test
# AppellF1(1,2,3,4,.25,.5)

In [26]:
# def m200_b(r200, k0, n, gamma, r_s, r_c):
#     return (1.68793e9 * r200**3 * r_c**-gamma * r_s**(-1 + gamma) * k0\
#             * AppellF1(3/2., gamma/2., (n - gamma)/2., 5/2., -(r200**2/r_c**2), -(r200**2/r_s**2)))
# # test
# m200_b(1470.22, 1.1, 3., 1.,  50.*4.333,15.*4.333)
# I abandoned this method because of convergence issues.

In [27]:
# set tolerance for relative error in integration to be reported
tol = 1.e-2

In [28]:
# trying numerical integral instead of AppellF1, due to convergence issues
# This version has no correction for axis ratio.
def m200_b1(r200_as, k0, n, gamma, r_s_as, r_c_as, dummy):  # note distances must be in arc seconds
    r200 = r200_as * kpc_per_arcsec.value
    r_c = r_c_as * kpc_per_arcsec.value
    r_s = r_s_as * kpc_per_arcsec.value
    res = quad(lambda r: 4. * np.pi * r**2 *  (k0 * sigma_crit *  r_s**n)\
                /( 2. * np.pi * r_s * (r**2 + r_c**2)**(gamma/2.) * (r**2 + r_s**2)**((n - gamma)/2.)), 0, r200)
    if res[0] !=0.:
        if res[0]> .01 and np.abs(res[1] / res[0]) > tol:
            print('Warning: relative error is above threshold.')
            print('Result = {0:1.6e}, Error = {1:1.6e}, relative error = {2:2%}'.format(res[0], res[1], res[1]/res[0]))
    return res[0]
# test
%time test = m200_b1(339, 1.1, 3., 1., 50., 15., 5e99)
print('mass = {0:1.5e}'.format(test))

CPU times: user 177 µs, sys: 6 µs, total: 183 µs
Wall time: 187 µs
mass = 3.05655e+14


In [29]:
# now trying numerical double integral incorporating axis ratio, to account for elliptical nature
def m200_b2(r200_as, k0, n, gamma, r_s_as, r_c_as, q):  # note distances must be in arc seconds
    r200 = r200_as * kpc_per_arcsec.value
    r_c = r_c_as * kpc_per_arcsec.value
    r_s = r_s_as * kpc_per_arcsec.value
    res = dblquad(lambda r, th: r**2 * np.sin(th) *  (k0 * sigma_crit *  r_s**(n-1)\
                /( ((r * np.sin(th))**2 + r_c**2 + (r * np.cos(th))**2/ q**2)**(gamma/2.)\
                  * ((r * np.sin(th))**2 + r_s**2 + (r * np.cos(th))**2/ q**2)**((n - gamma)/2.)))\
                , 0, np.pi, lambda r: 0., lambda r: r200)
    if res[0] !=0.:
        if res[0]> .01 and np.abs(res[1] / res[0]) > tol:
            print('Warning: error is above threshold.')
            print('Result = {0:1.6e}, Error = {1:1.6e}, relative error = {2:2%}'.format(res[0], res[1], res[1]/res[0]))
    return res[0]
# test
%time test = m200_b2(339, 1.1, 3., 1., 50., 15., 1)
print('mass = {0:1.5e}'.format(test))

CPU times: user 18.4 ms, sys: 1.35 ms, total: 19.8 ms
Wall time: 19.3 ms
mass = 3.05655e+14


In [30]:
# now trying the approximation method
def m200_b3(r200_as, k0, n, gamma, r_s_as, r_c_as, q):  # note distances must be in arc seconds
    r200 = r200_as * kpc_per_arcsec.value 
    r_c = r_c_as * kpc_per_arcsec.value * np.sqrt(q)
    r_s = r_s_as * kpc_per_arcsec.value * np.sqrt(q)
    # Manoj thought a factor of q**(-1./3) helped here, but I don't see that.
    res = quad(lambda r:  4. * np.pi * r**2 *  (k0 * sigma_crit *  r_s**n)\
                /( 2. * np.pi * r_s * (r**2 + r_c**2)**(gamma/2.) * (r**2 + r_s**2)**((n - gamma)/2.)), 0, r200)
    if res[0] !=0.:
        if res[0]> .01 and np.abs(res[1] / res[0]) > tol:
            print('Warning: relative error is above threshold.')
            print('Result = {0:1.6e}, Error = {1:1.6e}, relative error = {2:2%}'.format(res[0], res[1], res[1]/res[0]))
    return res[0]
# test
%time test = m200_b3(339, 1.1, 3., 1., 50., 15., 1)
print('mass = {0:1.5e}'.format(test))

CPU times: user 287 µs, sys: 2 µs, total: 289 µs
Wall time: 292 µs
mass = 3.05655e+14


In [31]:
4/3. * np.pi * 339**3. * rho_200

1.6797e+14

In [32]:
# find the root where the masses are equal
def find_virial(k0, n, gamma, r_s, r_c, q):  # r_c and r_s should be in arc seconds
    r = minimize_scalar(lambda r200: np.abs(m200_b1(r200, k0, n, gamma, r_s, r_c, q) - 4/3. * np.pi * r200**3.\
                        * rho_200), method = 'bounded', bounds=(0, 5000))  # r200 in units of arc seconds
    # according to Manoj, don't have to do any correction now.
    if r.success!=True:
        print('Optimize failure.')
        print(r)
        print('k0: {0:1.5f}  gamma: {1:1.5f}  r_s:{2:1.5f}  r_c:{3:1.5f}  q:{4:1.5f}'.format(k0, gamma, r_s, r_c, q))
    return r.x, 4/3. * np.pi * r.x**3. * rho_200
# test
%time print('virial radius = {0:1.2f}, virial mass = {1:1.5e}'.format(*find_virial(1.34, 3., 1.858, 17.7, .67, .8)))
%time print('virial radius = {0:1.2f}, virial mass = {1:1.5e}'.format(*find_virial(0.167, 3., 2.353, 133.656, 19.21, .8)))

virial radius = 265.44, virial mass = 8.06321e+13
CPU times: user 13.1 ms, sys: 370 µs, total: 13.5 ms
Wall time: 13.4 ms
virial radius = 433.71, virial mass = 3.51734e+14
CPU times: user 6.18 ms, sys: 193 µs, total: 6.38 ms
Wall time: 6.2 ms


In [33]:
# Now need a function to find c_hat. First, the Corecusp log slope.
def cclogslope(r, gamma, r_s, r_c): # assumes n=3 and rho_0 = 1.
    r2 = r * r
    rs2 = r_s * r_s
    rc2 = r_c * r_c
    return - (r2 * (3 * r2 - rc2 * (gamma-3) + gamma * rs2)) / ((r2 + rc2) * (r2 + rs2))
# test:
cclogslope(15, 1, 15, 0)

-2

In [34]:
# A function to find r_{beta}, the radius at which the log slope is 'beta'.
def rbeta(gamma, r_s, r_c, beta):
    rb = minimize_scalar(lambda r: np.abs(beta - cclogslope(r, gamma, r_s, r_c)), method='bounded', bounds=(0,5000))
    if rb.success !=True:
        print('Optimize Failure')
        print(rb)
        print(gamma, r_s, r_c, beta)
    return rb.x
# test
rbeta(1.61, 104.6, 12.7, -2) # should be 71.1519 per Mathematica

71.152

In [35]:
# non-parallel version, works
mvir = []
c_hat = []
r_core = []
rm2 = []
ns = len(k0)
n = np.ones(ns) * np.array([3.])  # n=3

In [36]:
for i, (k0p, gammap, npr, r_cp, r_sp, qp) in enumerate(zip(k0, gamma, n, r_c, r_s, q)):
    res = find_virial(k0p, npr, gammap, r_sp, r_cp, qp)
    mvir.append(res[1])
    rm2p = rbeta(gammap, r_sp, r_cp, -2)
    rm2.append(rm2p)
    c_hat.append(res[0] / rm2p)
    r_core.append(rbeta(gammap, r_sp, r_cp, -1))
    sys.stdout.write('\r{0:5.1%} complete.'.format(float(i) / ns))
        
sys.stdout.write("\rCalculation is complete.\n")

Calculation is complete.


In [37]:
# beep
duration = 0.2  # second
freq = 440  # Hz
os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

0

In [38]:
# # parallel version - need to fix to give r_core, rm2
# def virial_and_rbeta(k0, n, gamma, r_s, r_c, q, beta):
#     rv, mv = find_virial(k0, n, gamma, r_s, r_c, q)
#     rb = rbeta(gamma, r_s, r_c, beta)
#     return rv, mv, rb
# pool = mp.Pool(processes=nproc)
# ns = len(k0)
# mvir = np.zeros(ns)
# rvir = np.zeros(ns)
# c_hat = np.zeros(ns)
# results = []
# for i, (k0p, gammap, r_sp, r_cp, qp) in enumerate(zip(k0, gamma, r_s, r_c, q)):
#     results.append((i, pool.apply_async(virial_and_rbeta, args=(k0p, 3., gammap, r_sp, r_cp, qp, -2,))))
# print("Started.")
# pool.close()   # now nothing else can be added to the pool
# output = []
# tstart = timeit.default_timer()
# old_i = -1
# for (i,k) in results:
#     output.append(k.get())
#     (result0, result1, result2) = output[-1]
#     mvir[i] = result1
#     rvir[i] = result0
#     c_hat[i] = result0 / result2
#     if old_i +1 != i:
#         print('Out of order at i=', i)
#     old_i = i
#     tavg = (timeit.default_timer() - tstart) / (i+1.)
#     proj_fin =  tavg * (ns -i -1)
#     sys.stdout.write("\r{0:5.2%} complete. Avg time per iteration: {1:6.2f} sec. Projected finish in {2:4.2f} min."\
#                      .format((float(i) / ns), tavg,  proj_fin/60))
# sys.stdout.write("\rCalculation is complete.                     \n")

In [39]:
samples2[:, p-1:p-1+addl_np] = np.array([mvir, c_hat, r_core, rm2]).T

In [40]:
# confirm the correct value for the last sample
ind = -1
vir_test = find_virial(samples2[ind, 1], 3., *samples2[ind,2:6])[1]
if vir_test != samples2[ind, -4]:
    print(vir_test, samples2[ind, -3])

446358542791823.7 14.894334222673423


In [41]:
samples2[-1]

array([ 7.28790000e-06,  2.34949000e+00,  6.58374000e-01,  3.60380000e+01,
        1.11954000e-04,  6.21670000e-01,  2.79472000e+01,  4.30655000e-01,
        2.33500000e+00,  3.89054000e-03, -3.13350000e-02,  6.52295000e+12,
        2.12861000e+12,  1.45178000e+01,  4.46358543e+14,  1.12488890e+01,
        1.48943342e+01,  4.17423052e+01, -5.73624000e+00])

In [42]:
samples2.shape

(120367, 19)

In [43]:
# write the revised samples back to the file
np.savetxt(newlabel, samples2, fmt="%.6g", delimiter='   ')

In [44]:
for i, name in enumerate(paramnames):
    if name == 'raw_chisq':
        print('Best fit chi squared is ', np.amin(samples2[:,i+1]))

Best fit chi squared is  7.72813


In [45]:
newlabel

'A2537.cc.v9_d7'

In [46]:
sys.exit()

SystemExit: 

/Users/kevin/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [47]:
# If there is a transform that you want to make, and a dat file for conversion, specify it here
transform =True
t_file = 'v8_transform.dat'

In [48]:
# make the error file using mkdist
if transform:
    code = os.system('mkdist ' + newlabel + ' -E -T:../' + t_file + ' | tee ' + newlabel + '.err')
else:
    code = os.system('mkdist ' + newlabel + ' | tee ' + newlabel + '.err')    
if code != 0:
    print('Error! Code returned ', code)
else:
    print('Success')

Success


In [49]:
# make the various distributions and plots
# having trouble with 'ranges much smaller than actual ranges'
# so try adding the option '-B5e-4'
if transform:
    code = os.system('mkdist ' + newlabel + ' -n40 -N40  -B5e-4 -T:../' + t_file + ' |tee tmp.txt')
else:
    code = os.system('mkdist ' + newlabel + ' -n40 -N40  -B5e-4'+ ' |tee tmp.txt')
if code != 0:
    print('Error! Code returned ', code)
else:
    with open('tmp.txt', 'r') as file:
        tmp = file.readlines()
    if 'WARNING' in tmp or 'Error:' in tmp:
        print('Completed with WARNINGS or Error')
    elif len(tmp) == 1:
        print('Warning: output was only one line:')
        print(tmp)
    else:
        print('Success')

Success


In [ ]:
sys.exit()

In [50]:
# to run the python triangle plot pdf maker
code = os.system('python ' + newlabel + '_tri.py')
if code != 0:
    print('Error! Code returned ', code)
else:
    print('Success')

Success
